In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import keras
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn import metrics
from keras import models
from keras import layers
from keras.applications.vgg16 import VGG16
from keras.preprocessing.image import load_img, img_to_array, ImageDataGenerator
from sklearn.preprocessing import LabelEncoder

In [2]:
# Data gathering

x = []
y = []
IM_SIZE = 150
NUM_CLASSES = 120
path = './Data/dog-breed-identification'

df = pd.read_csv(f'{path}/labels.csv', header=0)
breeds = list(df.breed.value_counts()[:NUM_CLASSES].index)
labels = df[df.breed.isin(breeds)]

for root, _, files in os.walk(f'{path}/train/'):
    for file in files:
        if file.split(".")[0] in labels.id.values:
            img = load_img(f'{root}/{file}', grayscale=False, color_mode='rgb', target_size=(IM_SIZE, IM_SIZE))
            x.append(img_to_array(img))
            y.append(labels[labels.id == file.split(".")[0]].breed)

x = np.array(x)

In [3]:
# Label Encoding
le = LabelEncoder() 
y = le.fit_transform(y)

C:\Users\tim_a\anaconda3\envs\DLmisc\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


In [4]:
mapping = dict(zip(range(len(le.classes_)), le.classes_))

In [5]:
print(mapping)

{0: 'affenpinscher', 1: 'afghan_hound', 2: 'african_hunting_dog', 3: 'airedale', 4: 'american_staffordshire_terrier', 5: 'appenzeller', 6: 'australian_terrier', 7: 'basenji', 8: 'basset', 9: 'beagle', 10: 'bedlington_terrier', 11: 'bernese_mountain_dog', 12: 'black-and-tan_coonhound', 13: 'blenheim_spaniel', 14: 'bloodhound', 15: 'bluetick', 16: 'border_collie', 17: 'border_terrier', 18: 'borzoi', 19: 'boston_bull', 20: 'bouvier_des_flandres', 21: 'boxer', 22: 'brabancon_griffon', 23: 'briard', 24: 'brittany_spaniel', 25: 'bull_mastiff', 26: 'cairn', 27: 'cardigan', 28: 'chesapeake_bay_retriever', 29: 'chihuahua', 30: 'chow', 31: 'clumber', 32: 'cocker_spaniel', 33: 'collie', 34: 'curly-coated_retriever', 35: 'dandie_dinmont', 36: 'dhole', 37: 'dingo', 38: 'doberman', 39: 'english_foxhound', 40: 'english_setter', 41: 'english_springer', 42: 'entlebucher', 43: 'eskimo_dog', 44: 'flat-coated_retriever', 45: 'french_bulldog', 46: 'german_shepherd', 47: 'german_short-haired_pointer', 48: '

In [6]:
# Data splitting

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25)

In [7]:
y_train = keras.utils.to_categorical(y_train, num_classes=NUM_CLASSES)
y_test = keras.utils.to_categorical(y_test, num_classes=NUM_CLASSES)

In [8]:
# Data generators

train_datagen = ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.25,
    height_shift_range=0.25,
    rescale=1./255,
    shear_range=0.25,
    zoom_range=0.25,
    horizontal_flip=True,
    fill_mode='nearest')

test_datagen=ImageDataGenerator(rescale=1./255)

training_set=train_datagen.flow(x_train,y=y_train,batch_size=64)
testing_set=test_datagen.flow(x_test,y=y_test,batch_size=64)

In [9]:
# Model with pretrained base (VGG16)
from keras.layers import Flatten, Dense
from keras.models import Sequential

conv_base = VGG16(weights='imagenet',
                  include_top=False,
                  input_shape=(IM_SIZE, IM_SIZE, 3))

model = models.Sequential()

model.add(conv_base)
model.add(layers.Flatten())
model.add(layers.Dense(NUM_CLASSES, activation='softmax'))
conv_base.trainable = False
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Functional)           (None, 4, 4, 512)         14714688  
_________________________________________________________________
flatten (Flatten)            (None, 8192)              0         
_________________________________________________________________
dense (Dense)                (None, 120)               983160    
Total params: 15,697,848
Trainable params: 983,160
Non-trainable params: 14,714,688
_________________________________________________________________


In [ ]:
from keras import optimizers

model.compile(loss='categorical_crossentropy',
              optimizer="adam",
              metrics=['acc'])

history = model.fit(training_set, validation_data=testing_set, epochs=200, callbacks=[keras.callbacks.EarlyStopping(monitor="val_loss", patience=5)])

Epoch 1/200
120/120 [==============================] - 423s 3s/step - loss: 5.1511 - acc: 0.0286 - val_loss: 4.2401 - val_acc: 0.1045
Epoch 2/200
120/120 [==============================] - 246s 2s/step - loss: 4.1802 - acc: 0.1079 - val_loss: 3.9694 - val_acc: 0.1256
Epoch 3/200
120/120 [==============================] - 246s 2s/step - loss: 3.8384 - acc: 0.1557 - val_loss: 4.0424 - val_acc: 0.1393
Epoch 4/200
120/120 [==============================] - 246s 2s/step - loss: 3.7197 - acc: 0.1754 - val_loss: 3.9018 - val_acc: 0.1616
Epoch 5/200
120/120 [==============================] - 247s 2s/step - loss: 3.5518 - acc: 0.1958 - val_loss: 3.8603 - val_acc: 0.1741
Epoch 6/200
120/120 [==============================] - 246s 2s/step - loss: 3.4310 - acc: 0.2164 - val_loss: 3.7215 - val_acc: 0.1890
Epoch 7/200
120/120 [==============================] - 246s 2s/step - loss: 3.3317 - acc: 0.2325 - val_loss: 3.7889 - val_acc: 0.1866
Epoch 8/200
120/120 [==============================] - 246s 2s

In [ ]:
# Accuracy plot

plt.plot(history.history['acc'], 'go', label='Training acc', alpha=0.5)
plt.plot(history.history['val_acc'], 'r', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()
plt.show()

In [ ]:
model.save("pretrained_dbc.h5")